In [6]:
## Import necessary packages

#%matplotlib qt
# On Macs use osx
# For Windows use qt

import numpy as np
from numpy.random import rand
from landscapeWithOcean import LandscapeWithOcean # Import methods from inside file landscapeWithOcean.py

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
print('done')

ModuleNotFoundError: No module named 'PyQt4'

In [ ]:
def AddHill(Z,NX,NY,xx,yy,r,h):

    for x in range(NX):
        for y in range(NY):
            dx = np.mod(x-xx+NX/2,NX)-NX/2; # difference i-i0 but apply p.b.c. 
            dy = np.mod(y-yy+NY/2,NY)-NY/2;
            dr = np.sqrt(dx**2+dy**2);
            if (dr<r):
                Z[x,y] += h * (np.cos(dr/r*np.pi/2.0))**2;

    return Z

In [ ]:
### Define simulation grid and initial conditions

NX = 70 #number of rows
NY = 70 #number of columns

d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

# small random features in topography to begin erosion
Z = rand(NX,NY)*0.1
for i in range(5):
    xx = rand()*NX
    yy = rand()*NY
    r  = (0.1+rand())*NX
    h  = (0.1+rand())*5
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(5):
    xx = rand()*NX
    yy = rand()*NY
    r  = (0.1+rand())*NX/2
    h  = (0.5+rand())*10
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!
ZMaxOrg = np.max(Z)
#print(ZMaxOrg)

In [ ]:
### Physical Parameters
K = 1.0e-6 # meters^(1-2m)/yr

D = 0.005 # m^2/yr

# uplift rate
# uplift = 0.03 / 600.
uplift = 0.0

In [ ]:
### Model parameters

# Time step
dt = d**2 / D / 8. 
#dt = d**2 / D /16. #extra small steps 
print(' dt[years] = ',dt)

#Area exponent A^m, default m=1
m=1

#gradient exponent g^n, default n=1
n=1

#erosion threshold 
theta_c = 10 

In [ ]:
# Total simulation time
T = 1000.0 * 625.0
#T=100000.0*625.0
# total number of iterations
n_iter = int(np.round(T/dt))
print('Number of interation: ',n_iter)

In [ ]:
# Initialize landscape 
ls = LandscapeWithOcean(NX,NY)

oceanLevelParameter=0.1  # what does this parameter do?
ls.ComputeOceanVolumeFromOceanLevelParameter(Z,NX,NY,oceanLevelParameter)

ls.pool_check(Z,NX,NY)
ls.A = np.zeros((NX,NY))

In [2]:
# Set-up figure
def init_figure():
    fig = plt.figure(figsize=(12.,6.))
    plt.show()
    return fig

def update_figure():
        plt.clf()
        ax1 = fig.add_subplot(121,projection='3d')

        # use equal x-y aspect with an explicit vertical exageration
        vert_exag = 4.
        ax1.set_xlim3d(0,max(NX,NY))
        ax1.set_ylim3d(0,max(NX,NY))
        ax1.set_zlim3d(0,ZMaxOrg)

        ax1.set_title('Surface Relief')

#        surf = ax1.plot_surface(X,Y,Z, cmap = cm.terrain, rstride=1, cstride=1,
#                antialiased=False,linewidth=0)

        ZPlot = np.copy(Z)
        ZPlot[ZPlot<ls.ZBeachLevel] = ls.ZBeachLevel 
        ZPlot -= ls.ZBeachLevel
        ax1.plot_surface(X,Y,ZPlot, cmap = cm.terrain, rstride=1, cstride=1,
                antialiased=False,linewidth=0)

        ax2 = fig.add_subplot(122,aspect='equal')
        ax2.set_title('Elevation')

        #im = ax2.pcolor(Z,cmap=cm.terrain)
        im = ax2.pcolor(ZPlot,cmap=cm.coolwarm)
        cs = ax2.contour(ZPlot,6,colors='k')

        # Add a color bar which maps values to colors.
        cbar = fig.colorbar(im, shrink=0.5, aspect=5)
        # Add the contour line levels to the colorbar
        cbar.add_lines(cs)

        #plt.show()
        #plt.draw()
        plt.pause(0.0001)

In [3]:
# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

for it in range(1,n_iter+1):
    
    ls.calculate_collection_area(Z,NX,NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
        iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

        for j in range(NY):
            jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
            jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.
  
            if ls.ocean[i,j]>0:
                Psi_z  = 0;
                Phi_z  = 0;
            else:
                if ls.drain[i,j]>0: #this cell is a drain
                    s1 = (Z[iR,j]  - Z[iL,j] )/(2.*dx)
                    s2 = (Z[i,jU]  - Z[i,jD] )/(2.*dy)
                    s3 = (Z[iR,jD] - Z[iL,jU])/(2. * np.sqrt( dx**2 + dy**2) )
                    s4 = (Z[iR,jU] - Z[iL,jD])/(2. * np.sqrt( dx**2 + dy**2) )
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)            

                elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)
                
                    if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                        gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                    elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[iL,j])/dx
                    elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jU])/dy
                    elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jD])/dy
                    else:
                        gradient = 0.02 # ??? This does happen (maybe when two pools merge)
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)
                
                else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                    Psi_z = 0.
                
                if (Psi_z<0):
                    Psi_z = 0. 

                # diffusion term
                Phi_z = D * ( (Z[iR,j] - 2.*Z[i,j] + Z[iL,j]) / dx**2 \
                            + (Z[i,jU] - 2.*Z[i,j] + Z[i,jD]) / dx**2 )
           
            Znew[i,j] = Z[i,j] + (Phi_z - Psi_z + uplift )*dt  

            dZdt= (Znew[i,j] - Z[i,j]) / dt
            CFL = abs(dZdt) * dt / min(dx,dy)
            if (CFL>1.0):
                print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)
            
            if (Znew[i,j]<0.):
                Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero
    
    #Znew[0,:] = 0.0 # resets front boundary to 0
    Z = np.copy(Znew)
    
    ls.pool_check(Z,NX,NY)

    if (np.mod(it,10)==0): 
        print(it,end='')
        update_figure()
        print(' Ocean level=',ls.ZBeachLevel,' Ocean surface fraction=',100*ls.AOcean/(NX*NY));
    else:
        print('.',end='')

update_figure()
print(' Simulation finished.')


NameError: name 'plt' is not defined

The geoligical features represent hills in the ocean. This code creates 2 randomly placed hills (Z), with a random radius (r), at a random height (h). 

The variable oceanLevelParameter represents the intial conditions of the water level.

Overtime the land erodes, causing the elevation to drop. 
The water responsed by increasing its level and surface area.*

*Based on my memory of the results. My labtop is unable to do these graphics. I didn't have time to write these comments while I   was in the lab and the results from running this are no longer shown.

In [ ]:
### Define simulation grid and initial conditions

NX = 70 #number of rows
NY = 70 #number of columns

d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

# small random features in topography to begin erosion
Z = rand(NX,NY)*0.1
#for i in range(5):
    #xx = rand()*NX
    #yy = rand()*NY
    #r  = (0.1+rand())*NX
    #h  = (0.1+rand())*5
    #Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 5+i
    yy = 5
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)
    
for i in range(60):
    xx = 5
    yy = 5+i
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 5+i
    yy = 60
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 60
    yy = 5+i
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(5):
    xx = 35
    yy = 35
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=20
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

    


x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!
ZMaxOrg = np.max(Z)
#print(ZMaxOrg)
print('done')

In [ ]:
# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

for it in range(1,n_iter+1):
    
    ls.calculate_collection_area(Z,NX,NY)
    ls.A *= dx*dy
    
    for i in range(NX):
        iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
        iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

        for j in range(NY):
            jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
            jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.
  
            if ls.ocean[i,j]>0:
                Psi_z  = 0;
                Phi_z  = 0;
            else:
                if ls.drain[i,j]>0: #this cell is a drain
                    s1 = (Z[iR,j]  - Z[iL,j] )/(2.*dx)
                    s2 = (Z[i,jU]  - Z[i,jD] )/(2.*dy)
                    s3 = (Z[iR,jD] - Z[iL,jU])/(2. * np.sqrt( dx**2 + dy**2) )
                    s4 = (Z[iR,jU] - Z[iL,jD])/(2. * np.sqrt( dx**2 + dy**2) )
                    gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)            

                elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)
                
                    if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                        gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                    elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[iL,j])/dx
                    elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jU])/dy
                    elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                        gradient = (Z[i,j]-Z[i,jD])/dy
                    else:
                        gradient = 0.02 # ??? This does happen (maybe when two pools merge)
                    Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)
                
                else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                    Psi_z = 0.
                
                if (Psi_z<0):
                    Psi_z = 0. 

                # diffusion term
                Phi_z = D * ( (Z[iR,j] - 2.*Z[i,j] + Z[iL,j]) / dx**2 \
                            + (Z[i,jU] - 2.*Z[i,j] + Z[i,jD]) / dx**2 )
           
            Znew[i,j] = Z[i,j] + (Phi_z - Psi_z + uplift )*dt  

            dZdt= (Znew[i,j] - Z[i,j]) / dt
            CFL = abs(dZdt) * dt / min(dx,dy)
            if (CFL>1.0):
                print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)
            
            if (Znew[i,j]<0.):
                Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero
    
    #Znew[0,:] = 0.0 # resets front boundary to 0
    Z = np.copy(Znew)
    
    ls.pool_check(Z,NX,NY)

    if (np.mod(it,10)==0): 
        print(it,end='')
        update_figure()
        print(' Ocean level=',ls.ZBeachLevel,' Ocean surface fraction=',100*ls.AOcean/(NX*NY));
    else:
        print('.',end='')

update_figure()
print(' Simulation finished.')


I modeled an island surrounded by a thinner rectangular island. Its unlikey that something like this would exist in nature but I thought It would be fun to model it and see how it erodes.

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm

In [ ]:
from matplotlib.animation import FFMpegWriter
metadata = dict(title='My first animation in 3D', artist='Matplotlib',comment='Wakanda is here now.')
writer = FFMpegWriter(fps=15, metadata=metadata,bitrate=200000)
fig = plt.figure(dpi=200)

In [9]:
### Define simulation grid and initial conditions

NX = 70 #number of rows
NY = 70 #number of columns

d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

# small random features in topography to begin erosion
Z = rand(NX,NY)*0.1
#for i in range(5):
    #xx = rand()*NX
    #yy = rand()*NY
    #r  = (0.1+rand())*NX
    #h  = (0.1+rand())*5
    #Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 5+i
    yy = 5
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)
    
for i in range(60):
    xx = 5
    yy = 5+i
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 5+i
    yy = 60
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 60
    yy = 5+i
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(5):
    xx = 35
    yy = 35
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=20
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

    


x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!
ZMaxOrg = np.max(Z)
#print(ZMaxOrg)
print('done')

done


Over time the elevated areas ar being eroded. The water level is increasing and the area of land above water is decreasing. The rectangular range of islands are eroding much faster than the center island. There is no uplift happening as it is set to 0. 

In [10]:
from matplotlib.animation import FFMpegWriter
metadata = dict(title='My first animation in 3D', artist='Matplotlib',comment='Wakanda is here now.')
writer = FFMpegWriter(fps=15, metadata=metadata,bitrate=200000)
fig = plt.figure(dpi=200)

# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

with writer.saving(fig, "wave2d.mp4", dpi=200):
    for it in range(1,n_iter+1):

        ls.calculate_collection_area(Z,NX,NY)
        ls.A *= dx*dy

        for i in range(NX):
            iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
            iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

            for j in range(NY):
                jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
                jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.

                if ls.ocean[i,j]>0:
                    Psi_z  = 0;
                    Phi_z  = 0;
                else:
                    if ls.drain[i,j]>0: #this cell is a drain
                        s1 = (Z[iR,j]  - Z[iL,j] )/(2.*dx)
                        s2 = (Z[i,jU]  - Z[i,jD] )/(2.*dy)
                        s3 = (Z[iR,jD] - Z[iL,jU])/(2. * np.sqrt( dx**2 + dy**2) )
                        s4 = (Z[iR,jU] - Z[iL,jD])/(2. * np.sqrt( dx**2 + dy**2) )
                        gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                        Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)            

                    elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)

                        if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                            gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                        elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[iL,j])/dx
                        elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[i,jU])/dy
                        elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[i,jD])/dy
                        else:
                            gradient = 0.02 # ??? This does happen (maybe when two pools merge)
                        Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)

                    else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                        Psi_z = 0.

                    if (Psi_z<0):
                        Psi_z = 0. 

                    # diffusion term
                    Phi_z = D * ( (Z[iR,j] - 2.*Z[i,j] + Z[iL,j]) / dx**2 \
                                + (Z[i,jU] - 2.*Z[i,j] + Z[i,jD]) / dx**2 )

                Znew[i,j] = Z[i,j] + (Phi_z - Psi_z + uplift )*dt  

                dZdt= (Znew[i,j] - Z[i,j]) / dt
                CFL = abs(dZdt) * dt / min(dx,dy)
                if (CFL>1.0):
                    print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)

                if (Znew[i,j]<0.):
                    Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero

        #Znew[0,:] = 0.0 # resets front boundary to 0
        Z = np.copy(Znew)

        ls.pool_check(Z,NX,NY)

        if (np.mod(it,10)==0): 
            print(it,end='')
            update_figure()
            print(' Ocean level=',ls.ZBeachLevel,' Ocean surface fraction=',100*ls.AOcean/(NX*NY));
        else:
            print('.',end='')
        
        plt.pause(0.01)
        writer.grab_frame()

update_figure()
print(' Simulation finished.')


.






































































































































































































































































































































































































































.
































































































































































































































































































































































































































.




































































































































































































































































































.


























































































































































































































































.














































































































































































































































...

10 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796























































.





















































.



































.































.





























.






















.




















.














.










.








20 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796






.




.


.
.


.....30 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........40 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........50 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........60 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........70 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........80 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........90 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........100 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........110 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........120 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........130 Ocean level= 0.553491866598496  Ocean surface fraction= 26.53061224489796
.........140 Ocean level= 0.553491866598496  Ocean sur

.........970 Ocean level= 0.5410831200534258  Ocean surface fraction= 51.02040816326531
.........980 Ocean level= 0.5410830759275411  Ocean surface fraction= 51.38775510204081
.........990 Ocean level= 0.5410830638837132  Ocean surface fraction= 51.69387755102041
.........1000 Ocean level= 0.5410830638837132  Ocean surface fraction= 52.06122448979592
 Simulation finished.


Over time the elevated areas ar being eroded. The water level is increasing and the area of land above water is decreasing

In [41]:
### Physical Parameters
#K = 1.0e-6 # meters^(1-2m)/yr
K = 1.0e-15
#D = 0.005 # m^2/yr
D = 0.5
# uplift rate
# uplift = 0.03 / 600.
uplift = 0.0
#uplift =0.03 / 600
### Model parameters

# Time step
#dt = d**2 / D / 8.
dt = (d**2 / D / 8.)*2
#dt = d**2 / D /16. #extra small steps 
print(' dt[years] = ',dt)

#Area exponent A^m, default m=1
m=1
#m=3
m=2
#gradient exponent g^n, default n=1
#n=1
n=2
#erosion threshold 
theta_c = 10 

 dt[years] =  12.5


In [42]:
### Define simulation grid and initial conditions

NX = 70 #number of rows
NY = 70 #number of columns

d  = 5 # grid spacing in meters
dx = d # keep dx=dy for simplicity
dy = d

LX=NX*dx
LY=NY*dy

# small random features in topography to begin erosion
Z = rand(NX,NY)*0.1
#for i in range(5):
    #xx = rand()*NX
    #yy = rand()*NY
    #r  = (0.1+rand())*NX
    #h  = (0.1+rand())*5
    #Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 5+i
    yy = 5
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)
    
for i in range(60):
    xx = 5
    yy = 5+i
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 5+i
    yy = 60
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(60):
    xx = 60
    yy = 5+i
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=5
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

for i in range(5):
    xx = 35
    yy = 35
    #yy=68
    #r  = (0.1+rand())*NX/2
    r=20
    #h  = (0.5+rand())*10
    h=50
    Z = AddHill(Z,NX,NY,xx,yy,r,h)

    


x = np.arange(NX)
y = np.arange(NY)
X,Y = np.meshgrid(y,x) #strange that y goes first !!!
ZMaxOrg = np.max(Z)
#print(ZMaxOrg)
print('done')

done


In [43]:
from matplotlib.animation import FFMpegWriter
metadata = dict(title='My first animation in 3D', artist='Matplotlib',comment='Wakanda is here now.')
writer = FFMpegWriter(fps=15, metadata=metadata,bitrate=200000)
fig = plt.figure(dpi=200)

# Set up figure
fig = init_figure()
update_figure()
Znew = np.copy(Z)

with writer.saving(fig, "wave2d_Final_Change2.mp4", dpi=200):
    for it in range(1,n_iter+1):

        ls.calculate_collection_area(Z,NX,NY)
        ls.A *= dx*dy

        for i in range(NX):
            iL = np.mod(i-1,NX) # normally i-1 but observe p.b.c.
            iR = np.mod(i+1,NX) # normally i+1 but observe p.b.c.

            for j in range(NY):
                jD = np.mod(j-1,NY) # normally j-1 but observe p.b.c.
                jU = np.mod(j+1,NY) # normally j+1 but observe p.b.c.

                if ls.ocean[i,j]>0:
                    Psi_z  = 0;
                    Phi_z  = 0;
                else:
                    if ls.drain[i,j]>0: #this cell is a drain
                        s1 = (Z[iR,j]  - Z[iL,j] )/(2.*dx)
                        s2 = (Z[i,jU]  - Z[i,jD] )/(2.*dy)
                        s3 = (Z[iR,jD] - Z[iL,jU])/(2. * np.sqrt( dx**2 + dy**2) )
                        s4 = (Z[iR,jU] - Z[iL,jD])/(2. * np.sqrt( dx**2 + dy**2) )
                        gradient = (np.sqrt(s1**2 + s2**2) + np.sqrt(s3**2 + s4**2))/2.
                        Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)            

                    elif ls.drainage[i,j]>0: #this cell is a drainage point (it drains a pool)

                        if (Z[i,j]>=Z[iR,j]) and ls.pool[iR,j]!=ls.drainage[i,j]: 
                            gradient = (Z[i,j]-Z[iR,j])/dx #pool is on my left, I drain to the right, use this gradiant
                        elif (Z[i,j]>=Z[iL,j]) and ls.pool[iL,j]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[iL,j])/dx
                        elif (Z[i,j]>=Z[i,jU]) and ls.pool[i,jU]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[i,jU])/dy
                        elif (Z[i,j]>=Z[i,jD]) and ls.pool[i,jD]!=ls.drainage[i,j]:
                            gradient = (Z[i,j]-Z[i,jD])/dy
                        else:
                            gradient = 0.02 # ??? This does happen (maybe when two pools merge)
                        Psi_z = K * ( ls.A[i,j]**m * gradient**n - theta_c)

                    else: #this cell is a pool, assume it has some mass diffusion but no erosion!
                        Psi_z = 0.

                    if (Psi_z<0):
                        Psi_z = 0. 

                    # diffusion term
                    Phi_z = D * ( (Z[iR,j] - 2.*Z[i,j] + Z[iL,j]) / dx**2 \
                                + (Z[i,jU] - 2.*Z[i,j] + Z[i,jD]) / dx**2 )

                Znew[i,j] = Z[i,j] + (Phi_z - Psi_z + uplift )*dt  

                dZdt= (Znew[i,j] - Z[i,j]) / dt
                CFL = abs(dZdt) * dt / min(dx,dy)
                if (CFL>1.0):
                    print('\nWarning: Time step of',dt,'is probably too large. Safer would be:',dt/CFL)

                if (Znew[i,j]<0.):
                    Znew[i,j] = 0. # yes, this does happen at the boundary when kept at zero

        #Znew[0,:] = 0.0 # resets front boundary to 0
        Z = np.copy(Znew)

        ls.pool_check(Z,NX,NY)

        if (np.mod(it,10)==0): 
            print(it,end='')
            update_figure()
            print(' Ocean level=',ls.ZBeachLevel,' Ocean surface fraction=',100*ls.AOcean/(NX*NY));
        else:
            print('.',end='')
        
        plt.pause(0.01)
        writer.grab_frame()

update_figure()
print(' Simulation finished.')











































.














































































































































































































































































































































































































































































































































































































.














































































































































































































































































































































































































































































































































































.




































































































































































































































































































































































































































.






































































































































































































































































































































































































































































.


























































































































































































































































































































































































































































































































...

.






























































.




























































.





















































.















































.



































.
























.
















20 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919












.










.




.
......30 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........40 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........50 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........60 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........70 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........80 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........90 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........100 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........110 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........120 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........130 Ocean level= 0.8015526273335837  Ocean surface fraction= 32.61224489795919
.........140 Ocean level= 0.801552

.........900 Ocean level= 0.42435090531618386  Ocean surface fraction= 75.42857142857143
.........910 Ocean level= 0.4243495893993635  Ocean surface fraction= 75.85714285714286
.........920 Ocean level= 0.4243484688985467  Ocean surface fraction= 76.06122448979592
.........930 Ocean level= 0.42434769391352317  Ocean surface fraction= 76.24489795918367
.........940 Ocean level= 0.42434713950338143  Ocean surface fraction= 76.14285714285714
.........950 Ocean level= 0.4243471147382786  Ocean surface fraction= 76.34693877551021
.........960 Ocean level= 0.42434680355425003  Ocean surface fraction= 76.42857142857143
.........970 Ocean level= 0.4243466117674445  Ocean surface fraction= 76.48979591836735
.........980 Ocean level= 0.4243453860593633  Ocean surface fraction= 75.93877551020408
.........990 Ocean level= 0.42434359474027644  Ocean surface fraction= 76.51020408163265
.........1000 Ocean level= 0.42434287373017504  Ocean surface fraction= 76.85714285714286
 Simulation finished.


Unfortunately the computer lab was closing while I was testing out the parameters and therefore I couldn't completely evaluate what the effects of each parameter is. I noticed however, that changing m or n had easy to see, drastic changes. When I increased m, everything eroded much quicker. Interestingly, when I changed only m, the rectangular islands eroded slower than the center island which was different from the rest. As they eroded, they became very steep. When I increased n, the rectangular islands shrunk really fast while the center was erodeding quicker than usual and with a greater slope. I did attempt to test what would happen when I changed only one of D,k,dt, and uplift, but I didn't see a significant change. This is likely due to not changing those variables enough.

After testing the variables, I changed all of the values for D,k,dt,m, and n and ran it. There wasn't a huge change with these new parameters but due to the computer lab closing, I left the modifications that I made. The most significant change to the figure once these variables were adjusted was that the center island eroded in a way that its radius appeared to be larger as time went on than it did before these changes were made.